# Look into SWAT test files

In [8]:
# 51 is all a's
# 61,71,81 is all a's, but high rate and few?

import sys
import ast

default_keyorder = { 27:0, 25:1, 23:2, 22:3, 20:4, 21:5, 26:6, 24:7 }

if  not hasattr(sys, 'ps1'):

    import argparse
    parser = argparse.ArgumentParser("Read the SWAT file and produce outputs")
    
    parser.add_argument('-o', '--obs-id', help="Observation ID to read")
    parser.add_argument('-c', '--calib', default=0, type=int, help="Apply calibration pickle file with name {calib}_calibs.pickle.\n"
                                                                   "  Default 0, to make calibration file from events with all telescopes hit.")
    parser.add_argument('-d', '--directory', default="./triggers/2025/06/0*/*", help="Directory in which to look for files (usually of form triggers/yyyy/mm/dd/).\n"
                                                                                     "  Uses glob, so *s accepted, if argument in quotes.")
    parser.add_argument('-k', '--keyorder', default="",help=f"Dictionary (as string) of the order of the tels in the TATS string.  e.g. {str(default_keyorder)}. Looks in file \"key_to_order.txt\" if not, or defaults this.")
    
    args = parser.parse_args()

    if args.obs_id:
        obsid =args.obs_id
    else:
        print("Need obsid!!!")
        sys.exit()
    
    calib = args.calib
    directory = args.directory
    if len(args.keyorder) == 0: # No keyprder provided on command line, try for file, and if no file default to default
        try:
            with open("key_to_order.txt", "r") as key_to_order_file:
                file_key_to_order = key_to_order_file.readline()
        except:
            print(f"No keyOrder file \"key_to_order.txt\", using default {str(default_keyorder)}.")
            key_to_order = default_keyorder
        else:
            try:
                key_to_order = ast.literal_eval(file_key_to_order)
            except:
                print("Error interpreting keyOrder string from \"key_to_order.txt\" as a dictionary:",file_key_to_order)
                print("Exiting...")
                sys.exit()
            print(f"Using keyOrder string from \"key_to_order.txt\" as a dictionary: {key_to_order}")

    else:
        try:
            key_to_order = ast.literal_eval(args.keyorder)
        except:
            print("Error interpreting keyOrder string argument as a dictionary:",args.keyorder)
            print("Exiting...")
            sys.exit()
    

else:
    obsid = 92 # 1301 # 97 # 92 # 1003 # 1004 # 1005
    calib = 61
    directory = "./triggers/2025/06/0*/*"
    key_to_order = default_keyorder

if len(key_to_order) != 8:
    print("Not enough channels in keyOrder dictionary",key_to_order)
    print("Exiting...")
    sys.exit()

In [2]:
import pickle
import pprint
if calib>0:
    with open(f"outputs/{calib}_calibs.pickle", 'rb') as handle:
        cal = pickle.load(handle)
    pprint.pprint(cal)

{20: 0.0,
 21: -0.6820988614511525,
 22: 0.1629704009729646,
 23: 0.254120098036641,
 24: 0.09889136270882502,
 25: -0.6311605584198015,
 26: -0.3750564932467086,
 27: 0.4122332312731099}


In [3]:
import numpy as np

In [4]:
from glob import glob
import re

In [5]:
from protozfits import File

In [6]:
files = glob(directory)
files.sort()
print(f"Number of files found in directory {directory}:",len(files))
files[:10],files[-10:]

Number of files found in directory ./triggers/2025/06/0*/*: 528


(['./triggers/2025/06/01/SUB000_SWAT000_20250602T081311_SBID0000000000000000000_OBSID0000000000000000000_SUBARRAY_CHUNK000.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T082642_SBID0000000000000000001_OBSID0000000000000000011_SUBARRAY_CHUNK000.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T082642_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK000.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T082858_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK001.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T083103_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK002.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T083301_SBID0000000000000000001_OBSID0000000000000000011_TEL_CHUNK003.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T083436_SBID0000000000000000001_OBSID0000000000000000012_SUBARRAY_CHUNK000.fits.fz',
  './triggers/2025/06/01/SUB002_SWAT001_20250602T083436_SBID0000000000000000001_OBSID0000

## Find the ObsIDs

In [7]:
re.findall(".+OBSID([0-9]+).+",files[0])

pattern = re.compile(r".+OBSID([0-9]+).+")

obsids = [int(pattern.findall(file)[0]) for file in files]
obsids = sorted(list(set(obsids)))
print(f"ObsIDs available in directory {directory}:\n",obsids)

ObsIDs available in directory ./triggers/2025/06/0*/*:
 [0, 11, 12, 21, 31, 41, 42, 51, 61, 71, 81, 91, 92, 93, 94, 95, 96, 97, 98, 101, 1001, 1002, 1003, 1004, 1005, 1101, 1201, 1301]


## Pick an ObsID

In [8]:
def get_files_for_obsid(obsid):
    #for file in files:
    #    print(file+"\n" if (re.findall(f".+OBSID(?:0+){obsid}_.+",file) != []) else "",end="")

    obsid_files = [file for file in files if (re.findall(f".+OBSID(?:0+){obsid}_.+",file) != []) ]
    #obsid_files
    
    ## Separate out the `SUBARRAY` files
    
    obsid_subarray = [file for file in obsid_files if "SUBARRAY" in file]
    print(f"{len(obsid_subarray):5d} subarray files")
    
    obsid_tel = sorted(list(set(obsid_files)-set(obsid_subarray)))
    print(f"{len(obsid_tel):} telescope files")

    return obsid_subarray,obsid_tel

## Look in the files

In [9]:
obsid_subarray,obsid_tel = get_files_for_obsid(obsid)

    1 subarray files
1 telescope files


In [10]:
f_tel = File(obsid_tel[0])
f_sub = File(obsid_subarray[0])

In [11]:
len(f_sub.SubarrayEvents),len(f_tel.Triggers)

(5295129, 22689969)

In [16]:
[(evt.event_id,evt.event_time_s,evt.event_time_qns//4,
  np.array((evt.tel_ids_with_trigger,evt.trigger_ids)).T) for evt in list(f_sub.SubarrayEvents[5931:5951])]

[(5931,
  1748939095,
  235923411,
  array([[        26, 2627739051],
         [        21, 2627740415],
         [        20, 2627740076]], dtype=uint64)),
 (5932,
  1748939095,
  235927358,
  array([[        25, 2569663054],
         [        23, 2627740196],
         [        27, 2627739698],
         [        24, 2627740096],
         [        21, 2627740416],
         [        26, 2627739052],
         [        22, 2627740470],
         [        20, 2627740077]], dtype=uint64)),
 (5933,
  1748939095,
  236176409,
  array([[        26, 2627739053],
         [        24, 2627740097],
         [        27, 2627739699]], dtype=uint64)),
 (5934,
  1748939095,
  236180379,
  array([[        20, 2627740078],
         [        23, 2627740199]], dtype=uint64)),
 (5935,
  1748939095,
  236328127,
  array([[        26, 2627739054],
         [        27, 2627739700],
         [        21, 2627740417],
         [        20, 2627740079],
         [        24, 2627740099],
         [        25, 

In [18]:
[(20000+i,tel.tel_id,tel.trigger_id,tel.trigger_time_s,tel.trigger_time_qns//4) for i,tel in enumerate(list(f_tel.Triggers[20000:20100]))]

[(20000, 27, 2627739059, 1748939095, 58249117),
 (20001, 24, 2627739366, 1748939095, 58370920),
 (20002, 21, 2627739666, 1748939095, 58370934),
 (20003, 26, 2627738531, 1748939095, 58370935),
 (20004, 21, 2627739667, 1748939095, 58417461),
 (20005, 26, 2627738532, 1748939095, 58417462),
 (20006, 20, 2627739342, 1748939095, 58417467),
 (20007, 25, 2569662601, 1748939095, 58454141),
 (20008, 21, 2627739668, 1748939095, 58499661),
 (20009, 20, 2627739343, 1748939095, 58499666),
 (20010, 20, 2627739344, 1748939095, 58517636),
 (20011, 21, 2627739669, 1748939095, 58517653),
 (20012, 22, 2627739659, 1748939095, 58623463),
 (20013, 24, 2627739367, 1748939095, 58623473),
 (20014, 23, 2627739446, 1748939095, 58623475),
 (20015, 21, 2627739670, 1748939095, 58623479),
 (20016, 20, 2627739345, 1748939095, 58628425),
 (20017, 25, 2569662602, 1748939095, 58635554),
 (20018, 23, 2627739447, 1748939095, 58635563),
 (20019, 27, 2627739060, 1748939095, 58635563),
 (20020, 24, 2627739368, 1748939095, 586

In [19]:
f_sub.SubarrayEvents[1],list(f_tel.Triggers[0:1010])

(EventWrapper(
     event_id=1
     trigger_type=1
     sb_id=9
     obs_id=92
     event_time_s=1748939094
     event_time_qns=1600306844
     trigger_ids=array([2569660913, 2627736703], dtype=uint64)
     tel_ids_with_trigger=array([25, 23], dtype=uint16)
     tel_ids_with_data=array([25, 23], dtype=uint16)),
 [TriggerWrapper(
      tel_id=22
      trigger_id=2627736737
      trigger_type=3
      trigger_time_s=1748939094
      trigger_time_qns=1600008440
      readout_requested=False
      data_available=True
      hardware_stereo_trigger_mask=0),
  TriggerWrapper(
      tel_id=23
      trigger_id=2627736702
      trigger_type=3
      trigger_time_s=1748939094
      trigger_time_qns=1600008452
      readout_requested=False
      data_available=True
      hardware_stereo_trigger_mask=189),
  TriggerWrapper(
      tel_id=25
      trigger_id=2569660913
      trigger_type=3
      trigger_time_s=1748939094
      trigger_time_qns=1600306844
      readout_requested=False
      data_availab

In [20]:
f_sub.SubarrayEvents[1].trigger_ids

array([2569660913, 2627736703], dtype=uint64)

In [24]:
if  hasattr(sys, 'ps1'):
    # Looking at outputs, SWAT takes some time to connect to all...
    f_tel = File(obsid_tel[0])
    f_sub = File(obsid_subarray[0])
    prev_sub_tel_ids_with_trigger = []
    prev_s,prev_qns = 0,0
    for sub in f_sub.SubarrayEvents[:10000]:
        if (len(sub.tel_ids_with_trigger) != len(prev_sub_tel_ids_with_trigger) or
            set(sub.tel_ids_with_trigger) != set(prev_sub_tel_ids_with_trigger)):
            print(sub.event_id,"Subarray tel_ids",sub.event_time_s,sub.event_time_qns,"time",sub.tel_ids_with_trigger)
            differ = set(sub.tel_ids_with_trigger.astype(int))-set(prev_sub_tel_ids_with_trigger)
            difftimes = (sub.event_time_s-prev_s)*1_000_000_000 + (sub.event_time_qns-prev_qns)//4
            print("  event difference", np.array(list(differ)),"times",difftimes)
            prev_sub_tel_ids_with_trigger = sub.tel_ids_with_trigger.astype(int)
        difftimes = (sub.event_time_s-prev_s)*1_000_000_000 + (sub.event_time_qns-prev_qns)//4
        print("  time difference",difftimes)
        prev_s,prev_qns = sub.event_time_s,sub.event_time_qns

0 Subarray tel_ids 1748939094 1600008440 time [22 23]
  event difference [22 23] times 1748939094400002110
  time difference 1748939094400002110
1 Subarray tel_ids 1748939094 1600306844 time [25 23]
  event difference [25] times 74601
  time difference 74601
2 Subarray tel_ids 1748939094 1600314188 time [24 22 23]
  event difference [24 22] times 1836
  time difference 1836
3 Subarray tel_ids 1748939094 1601028760 time [26 21 27]
  event difference [26 27 21] times 178643
  time difference 178643
4 Subarray tel_ids 1748939094 1601428896 time [21 26 20 27 24]
  event difference [24 20] times 100034
  time difference 100034
5 Subarray tel_ids 1748939094 1601896532 time [24 23]
  event difference [23] times 116909
  time difference 116909
6 Subarray tel_ids 1748939094 1602552088 time [24 20 27 26]
  event difference [26 27 20] times 163889
  time difference 163889
7 Subarray tel_ids 1748939094 1602585376 time [21 24 22]
  event difference [21 22] times 8322
  time difference 8322
8 Subarr

In [25]:
from sys import maxsize
maxsize/1e9/(24*60*60)/365.25
maxsize

9223372036854775807

## Loop through the files and produce outputs

In [26]:
ord_a = ord("a")

In [30]:
obsids

[0,
 11,
 12,
 21,
 31,
 41,
 42,
 51,
 61,
 71,
 81,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 101,
 1001,
 1002,
 1003,
 1004,
 1005,
 1101,
 1201,
 1301]

In [31]:
# Order
# Channels vs key
#   0,  1,  2,  3,  4,  5,  6,  7
#  27, 25, 23, 22, 20, 21, 26, 24

#key_to_order = { 27:0, 25:1, 23:2, 22:3, 20:4, 21:5, 26:6, 24:7 }


# MP revised version with test remotely, 20250701 ... not correct for files taken!!!
# Maybe the IPs are assigned in random order after boot ?
#key_to_order = { 22:0, 27:1, 21:2, 24:3, 20:4, 25:5, 26:6, 23:7 }

In [32]:
import gzip
file_times = gzip.open(f"outputs/{obsid}_times.txt.gz", "wt")
file_strings = gzip.open(f"outputs/{obsid}_strings_calib_{calib}.txt.gz", "wt")

obsid_subarray,obsid_tel = get_files_for_obsid(obsid)

f_tel = File(ft:=obsid_tel.pop(0))
f_sub = File(fs:=obsid_subarray.pop(0))
print(ft,fs)

# 2.5ns spacing between deltaTs of TATS
t_step = 2.5

# In the SubarrayEvents, there is the trigger_ids, tel_ids.
#  Run through those for an event, then go through the tels, to get the times.

n_line = 0
sub_gen = f_sub.SubarrayEvents
tel_gen = f_tel.Triggers

# Start with first telescope record
tel = next(tel_gen)
t0 = tel.trigger_time_s-1_000_000_000 # Make it a second earlier for margin
sub_prev_trigger_time = t0

tel_files = {}
t_prevs = {}
e_prevs = {}

n_calibs = 0
calibs = {}

#for sub in f_sub.SubarrayEvents:
while True:
    try:
        sub = next(sub_gen)
    except StopIteration:
        f_sub.close()
        if len(obsid_subarray) == 0:
            break # No more subarray files
        next_subarray = obsid_subarray.pop(0)
        print("Nextsubarray file:",next_subarray)
        f_sub = File(next_subarray)
        sub_gen = f_sub.SubarrayEvents
        sub = next(sub_gen)        
    
    #print(sub)
    sub_tel_ids_with_trigger=sub.tel_ids_with_trigger
    sub_trigger_ids = sub.trigger_ids
    sub_trigger_time = (sub.event_time_s-t0)*1_000_000_000 + sub.event_time_qns//4
    #print("Subarray tel_ids",sub_tel_ids_with_trigger)
    #print("Subarray trigger_ids",sub_trigger_ids)
    sub_times={}

    tels = np.stack((sub_tel_ids_with_trigger,sub_trigger_ids)).T.tolist()

    #print("tels",tels)
    
    while len(tels):
        tel_id = tel.tel_id
        tel_idx = [tel.tel_id,tel.trigger_id]

    
        #print("***",tel_idx)
    
        if tel_idx in tels:
            #print("found tel_idx",tel_idx)
            
            # Open file if not already open
            if tel_id not in tel_files:
                tel_files[tel_id] = gzip.open(f"outputs/{obsid}_tel_{tel_id}.intervals.gz","wt")
                t_prevs[tel_id] = 0
                e_prevs[tel_id] = 0

            t_now_ns = tel.trigger_time_s*1_000_000_000 + tel.trigger_time_qns//4
            e_now = tel.trigger_id
            busy_status = not tel.data_available

            delta_ns = t_now_ns - t_prevs[tel_id]
            delta_evt = e_now - e_prevs[tel_id]

            tel_files[tel_id].write(f"{delta_ns:10} {delta_evt:10} {"1" if busy_status else "0"} {e_now:20} {t_now_ns:20}\n")

            t_prevs[tel_id] = t_now_ns
            e_prevs[tel_id] = e_now
            
            tels.remove(tel_idx)
            sub_times[tel.tel_id] = (tel.trigger_time_s-t0)*4_000_000_000+tel.trigger_time_qns

            continue

        # Could have a tel which has an event number higher than the subarray
        # => need to skip to the next subarray???
        # But without moving forward in the telescope file
        if tel.tel_id in sub.tel_ids_with_trigger:
            if tel.trigger_id > sub_trigger_ids[np.argwhere(sub_tel_ids_with_trigger==tel.tel_id)[0,0]]:
                #print(20*"v"+" TRIGGER AFTER SUB-ARRAY ??? "+20*"v")
                #file_strings.write(20*"v"+" TRIGGER AFTER SUB-ARRAY ??? "+20*"v"+"\n")
                continue
        else:
            #print(20*">"+" NON-COINCIDENT EVENT ??? "+20*">")
            #file_strings.write(20*">"+" NON-COINCIDENT EVENT ??? "+20*">"+"\n")
            # Are these at the same time as a missing packet?
            pass

        #print(">>>",tel_idx,tel.trigger_time_s,tel.trigger_time_qns)

        try:
            tel = next(tel_gen)
        except StopIteration:
            f_tel.close()
            if len(obsid_tel) == 0:
                break # No more telescope files
            next_tel = obsid_tel.pop(0)
            print("Next telescope file:",next_tel)
            f_tel = File(next_tel)
            tel_gen = f_tel.Triggers
            tel = next(tel_gen)

    # Sort by channel
    sub_times = dict(sorted(sub_times.items()))
    
    
    # get differences and convert from quarter_ns to ns
    min_times = min(sub_times.values())
    for key in sub_times.keys():
        sub_times[key] -= min_times
        sub_times[key] //= 4
    #print(sub_times)

    min_chan = min(sub_times, key=sub_times.get)

    if calib>0:
        #print("<<<")
        #pprint.pprint(sub_times)
        # Run through excluding 1st channel (but that shouldn't make a difference if included)
        for key in list(sub_times.keys())[1:]:
            sub_times[key] -= cal[key] - cal[min_chan] 
        #print(">>>")
        #pprint.pprint(sub_times)
 
    # re-do get differences
    min_times = min(sub_times.values())
    for key in sub_times.keys():
        sub_times[key] -= min_times

    #if len(sub_times)==8:
    #    break

    if calib>0:
        #print("===")
        #pprint.pprint(sub_times)
        pass

    
    '''
    if max(sub_times.values())>9:
        print("... FAR DISTANT DELTA_T ...")
        print(sub_times)
    '''

    times_str = "........"
    lts = list(times_str)
    for key in sub_times.keys():
        lts[key_to_order[key]] = f"{sub_times[key]}"[-1]    
    times_str = "".join(lts)
    
    #print(trig_str)
    file_times.write(times_str+"\n")

    '''
    #chr(ord_a + np.array(list(sub_times.values()))//4)
    # +np.round((t4th_ns-min_time)/cnt_per_ns/clk_period).astype('int')
    trig_str = "........"
    lts = list(trig_str)
    for key in sub_times.keys():
        lts[key_to_order[key]] = chr(ord_a + np.floor_divide(sub_times[key],t_step).astype('int'))  
    trig_str = "".join(lts)

    # Now, check if there is a better solution, by looking for minimum
    nearest_grid_times = {}
    sub_times_values = np.array(list(sub_times.values()))
    rmsq_min = np.inf
    for delta in np.linspace(-t_step/2,t_step/2,21):
        grid = np.round((sub_times_values+delta)/t_step).astype('int')
        grid_diff = sub_times_values-grid*t_step
        rmsq = np.sqrt(np.sum(grid_diff**2))
        if rmsq < rmsq_min:
            rmsq_min, delta_min, grid_min = rmsq, delta, grid
        elif rmsq == rmsq_min:
            delta_min2 = delta
    trig_str_min = "........"
    ltsm = list(trig_str_min)
    stv_keys = list(sub_times.keys())
    for key,gm in zip(stv_keys,grid_min):
        ltsm[key_to_order[key]] = chr(ord('a')+gm)
    trig_str_min = "".join(ltsm)

    if trig_str != trig_str_min:
        if n_line<10000 or n_line%100_000==0:
            if delta_min > 0 or delta_min2 < 0:
                print("        >> minimum better than nearest:",trig_str_min,trig_str,"minimum from",delta_min,"--",delta_min2)
        trig_str = trig_str_min

    '''
    
    # Rounding tested to be the best solution
    trig_str = "........"
    lts = list(trig_str)
    for key in sub_times.keys():
        lts[key_to_order[key]] = chr(ord('a')+
                                     np.round((sub_times[key])/t_step).astype('int'))
    trig_str = "".join(lts)


    dt = sub_trigger_time - sub_prev_trigger_time
    #print(trig_str)
    file_strings.write(trig_str+f" {dt}\n")
    sub_prev_trigger_time = sub_trigger_time

    # Try making the calibration w.r.t. 0th channel
    zeroth_chan = list(sub_times.keys())[0]
    if calib==0:
        if len(sub_times)==8:
            for key in sub_times.keys():
                calibs[key] = calibs.get(key, 0) + sub_times[key]-sub_times[zeroth_chan]
            n_calibs += 1
    
    if n_line<10000:
        print(trig_str)
    elif n_line%100_000==0:
        print(trig_str,n_line)
    elif n_line>1_000_000_000:
        break
    #print(80*"-")
    n_line += 1

    1 subarray files
1 telescope files
./triggers/2025/06/02/SUB002_SWAT001_20250603T082418_SBID0000000000000000009_OBSID0000000000000000092_TEL_CHUNK000.fits.fz ./triggers/2025/06/02/SUB002_SWAT001_20250603T082418_SBID0000000000000000009_OBSID0000000000000000092_SUBARRAY_CHUNK000.fits.fz
..ba....
.ag.....
..cb...a
i....ca.
f...faai
..b....a
f...b.ha
...c.a.a
.....ba.
.eca....
a.m...a.
.....da.
b..a....
aadghe.c
......aa
a....cd.
.....ae.
i.....a.
d.....a.
.a.....c
...a...d
..aj....
acg.....
..da....
a.....d.
.a..o...
.....a.e
e....ca.
ae...ga.
..cb...a
dm....a.
.....aa.
ca......
..eak..f
..fggj.a
da......
a....a.i
..ag....
....g..a
a....cc.
ba......
...ma..n
g.....fa
....ag..
..ag....
....ag.d
e.hfaabd
aa.i...f
ab......
.....aa.
....hca.
g....da.
....a..d
....el.a
a....kka
a...b...
..aa....
d.....a.
.....ba.
...f...a
.....a.a
...ah...
.....ad.
...haf.i
k....aed
....ac..
fa......
i.a....g
...cai..
daf.....
c......a
eknl.aae
.....ah.
..fc...a
b....aa.
.a.....g
....ga.i
ae...aaa
a..d....

KeyboardInterrupt: 

In [33]:
obsid

92

In [34]:
sub_times[key]

0.2127102754801815

In [35]:
key_to_order[key]

0

In [36]:
sub_times_values_reordered

NameError: name 'sub_times_values_reordered' is not defined

In [ ]:
if calib==0:
    cal = {k:v/n_calibs for k,v in calibs.items()}
    print(n_calibs,cal)
    with open(f"outputs/{obsid}_calibs.pickle", 'wb') as handle:
        pickle.dump(cal, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [165]:
sub_times

{27: 4000000034350458964}

In [166]:
key_to_order

{27: 0, 25: 1, 23: 2, 22: 3, 20: 4, 21: 5, 26: 6, 24: 7}